# Final Model Prediction

In [2]:
import sys
sys.path.append('a/UltimeTradingBot/Crypto_backtest_tools')
import pandas as pd
import json
import time
import numpy as np
import urllib
#import ccxt
import  ccxt  
import random
from keras.models import load_model
import os
import asyncio
import tensorflow as tf

## Garbage collection

In [3]:
import gc
gc.collect()    

import warnings
warnings.filterwarnings('ignore')
PRERR=False
def prerr(err):
    if PRERR:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

## Gloabale variables

In [4]:
Normalization=None
Model_FileName='w15_NoVol_XcryptoAi_model.hdf5'
Normalization_File='w15_NoVol_Normalization.json'
Binance_USDT_HALAL = [
    "BTC/USDT",
    "LUNA/USDT",
    "ETH/USDT",
    "GMT/USDT",
    "UST/USDT",
    "SOL/USDT",
    "APE/USDT",
    "XRP/USDT",
    "IDEX/USDT",
    "AVAX/USDT",
    "DOT/USDT",
    "ADA/USDT",
    "JASMY/USDT",
    "TRX/USDT",
    "NEAR/USDT",
    "AXS/USDT",
    "GAL/USDT",
    "GALA/USDT",
    "SHIB/USDT",
    "ZIL/USDT",
    "ENS/USDT",
    "DOGE/USDT",
    "LTC/USDT",
    "EUR/USDT",
    "MANA/USDT",
    "DAR/USDT",
    "WAVES/USDT",
    "LAZIO/USDT",
    "ALICE/USDT",
    "ROSE/USDT",
    "ZEC/USDT",
    "ALGO/USDT",
    "GRT/USDT",
    "PSG/USDT",
    "SLP/USDT",
    "EOS/USDT",
    "PORTO/USDT",
    "ICP/USDT",
    "EGLD/USDT",
    "XMR/USDT",
    "KDA/USDT",
    "ETC/USDT",
    "MBOX/USDT",
    "OGN/USDT",
    "AR/USDT",
    "GLMR/USDT",
    "LOKA/USDT",
    "XLM/USDT",
    "MTL/USDT",
    "SNX/USDT",
    "PYR/USDT",
    "DASH/USDT",
    "CITY/USDT",
    "ASTR/USDT",
    "IOTA/USDT",
    "ACM/USDT",
    "BAR/USDT",
    "JUV/USDT",
    "SYS/USDT",
    "RVN/USDT",
    "MBL/USDT",
    "REN/USDT",
    "JST/USDT",
    "OMG/USDT",
    "ATM/USDT",
    "XEC/USDT",
    "STORJ/USDT",
    "ZRX/USDT",
    "SRM/USDT",
    "ICX/USDT",
    "API3/USDT",
    "ONT/USDT",
    "SKL/USDT",
    "MULTI/USDT",
    "QTUM/USDT",
    "COCOS/USDT",
    "VOXEL/USDT",
    "HIVE/USDT",
    "KP3R/USDT",
    "ATA/USDT",
    "STMX/USDT",
    "ADX/USDT",
    "HIGH/USDT",
    "NULS/USDT",
    "MLN/USDT",
    "YGG/USDT",
    "SC/USDT",
    "CKB/USDT",
    "TOMO/USDT",
    "STX/USDT",
    "FLUX/USDT",
    "DNT/USDT",
    "ORN/USDT",
    "PLA/USDT",
    "BADGER/USDT",
    "DF/USDT",
    "MOB/USDT",
    "LPT/USDT",
    "SCRT/USDT",
    "RAD/USDT",
    "NMR/USDT",
    "ELF/USDT",
    "TORN/USDT",
    "T/USDT",
    "QUICK/USDT",
    "LSK/USDT",
    "FIDA/USDT",
    "XNO/USDT",
    "BTG/USDT",
    "GHST/USDT",
    "EPS/USDT"
]

pair_list = Binance_USDT_HALAL


## Local Functions first

In [5]:
# Get list of all IDs on binance
def give_first_kline_open_stamp(interval, symbol, start_ts=1499990400000):
        '''
        Returns the first kline from an interval and start timestamp and symbol
        :param interval:  1w, 1d, 1m etc - the bar length to query
        :param symbol:    BTCUSDT or LTCBTC etc
        :param start_ts:  Timestamp in miliseconds to start the query from
        :return:          The first open candle timestamp
        '''

        url_stub = "http://api.binance.com/api/v1/klines?interval="

        #/api/v1/klines?interval=1m&startTime=1536349500000&symbol=ETCBNB
        addInterval   = url_stub     + str(interval) + "&"
        addStarttime  = addInterval   + "startTime="  + str(start_ts) + "&"
        addSymbol     = addStarttime + "symbol="     + str(symbol)
        url_to_get = addSymbol

        kline_data = urllib.request.urlopen(url_to_get).read().decode("utf-8")
        kline_data = json.loads(kline_data)

        return kline_data[0][0]

def get_crypto_metadata(pair_list):
    Binance_USDT_HALAL=pair_list
    ids = []
    #ids = all_ids()
    for halalpair in Binance_USDT_HALAL:
    #    print( halalpair.replace('/',''))
        ids.append(halalpair.replace('/',''))
    #print(ids)
    MetaData=pd.DataFrame(ids)
    MetaData["Pair"]=Binance_USDT_HALAL
    counters=0
    for this_id in ids:
        '''
        Find launch Week of symbol, start at Binance launch date 2017-07-14 (1499990400000)
        Find launch Day of symbol in week
        Find launch minute of symbol in day
        '''

        symbol_launch_week_stamp   = give_first_kline_open_stamp('1w', this_id, 1499990400000 )
        symbol_launch_day_stamp    = give_first_kline_open_stamp('1d', this_id, symbol_launch_week_stamp)
        symbol_launch_minute_stamp = give_first_kline_open_stamp('1m', this_id, symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_week_stamp"]=str(symbol_launch_week_stamp)
        MetaData.loc[counters,"launch_day_stamp"]=str(symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_minute"]=pd.to_datetime(symbol_launch_minute_stamp, unit='ms')

        counters += 1

        #print("Week stamp", symbol_launch_week_stamp)
        #print("Day  stamp", symbol_launch_day_stamp)
        #print("Min  stamp", symbol_launch_minute_stamp)
        print(counters,end=" ")
        #print(this_id, "launched", symbol_launch_minute_stamp )
    return MetaData
    #print("")
    
def instant_pair_data(pair="GMT/BUSD",exchange=ccxt.binance(),window=15):
    ex=exchange
    ticker = ex.fetch_ticker(pair)
    pair_current_price=ticker['info']['askPrice']
    #print(pair_current_price)

    ohlcv1m = ex.fetch_ohlcv(pair, '1m', limit=15)
    ohlcv5m = ex.fetch_ohlcv(pair, '5m', limit=15)
    ohlcv15m = ex.fetch_ohlcv(pair, '15m', limit=15)
    ohlcv1h = ex.fetch_ohlcv(pair, '1h', limit=15)
    ohlcv1d = ex.fetch_ohlcv(pair, '1d', limit=15)

    pair_data=pd.DataFrame()
    pair_data.loc[0,"price"]=float(pair_current_price)
    #minute
    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)]=ohlcv1m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)]=ohlcv1m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)]=ohlcv1m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)]=ohlcv1m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)]=ohlcv1m[-window_i][4]

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_day"]=ohlcv1d[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_day"]=ohlcv1d[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_day"]=ohlcv1d[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_day"]=ohlcv1d[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_day"]=ohlcv1d[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_15min"]=ohlcv5m[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][4]   
    return  pair_data

def normalize(dataset,file='w15_NoVol_Normalization.json'):
    global Normalization
    try:
        N=Normalization
    except:
        Normalization=None
    if(Normalization==None):
        #print('Loading normalization from file')
        with open(file) as json_file:
            Normalization = json.load(json_file)
    else:
        #print('normalization is loaded')
        pass

    mean=np.array(Normalization["mean"])
    std=np.array(Normalization["std"])
    dataset -= mean 
    dataset /= std
    return(dataset)

def Buy_Dessision(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Normalized(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Multi_In_Out(input):
    A=np.array(input)
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

def Buy_Dessision_Multi_In_Out_Normalized(input):
    A=np.array(input)
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

# Getting Vars

In [6]:
try:
    MetaData=pd.read_csv("D:\+DATA+\MetaData.csv")
except:
    MetaData=get_crypto_metadata(Binance_USDT_HALAL)


## loding the model

In [7]:
model = load_model(Model_FileName)

In [8]:
Exchange=ccxt.binance()
ex=Exchange

In [9]:
pair='GMT/BUSD'
# bitcoin_ticker = Exchange.fetch_ticker('BTC/USDT')
# btc_current_price=bitcoin_ticker['info']['askPrice']
# print(btc_current_price)




# li1 = df.keys().values.tolist()
# li2 = fulldata.keys().values.tolist()
 
# s = set(li2)
# temp3 = [x for x in li1 if x not in s]
# print(temp3)

In [10]:
def instant_full_data(pair,exchange=ex,window=15):
    pdata=instant_pair_data(pair,exchange=exchange,window=15)
    btcdata=instant_pair_data("BTC/USDT",exchange=exchange,window=15).add_prefix("BTC_")
    Timestamp=pd.to_datetime(ex.fetchTime(),unit='ms')
    pdata=pd.concat([pdata,btcdata],axis=1)
    pdata.loc[0,"day"]=Timestamp.dayofweek+1
    pdata.loc[0,"hour"]=Timestamp.hour
    pdata.loc[0,"minute"]=Timestamp.minute
    pdata.loc[0,"lunch_day"]=int(-(pd.to_datetime(MetaData[MetaData["Pair"] == (pair.split("/")[0]+"/USDT")]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)
    for key in pdata.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            pdata[key]=(pdata["BTC_price"]-pdata[key])/pdata["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            pdata[key]=(pdata["price"]-pdata[key])/pdata["price"]
    return pdata

In [12]:

GoodDeal={}

while True:
    pdata=instant_full_data(pair,exchange=ex,window=15)
    if Buy_Dessision(pdata.iloc[0]) == 1:
        print("buying at"+str(pdata["price"].iloc[0]))
        GoodDeal[pdata["price"].iloc[0]]=pd.to_datetime(exchange.fetchTime(),unit='ms')
        pp=0
        for i in range(14):
            time.sleep(30)
            pp=ex.fetch_ticker(pair).ticker['info']['askPrice']
            if(float(pdata.loc[0,"price"]) >= float(pp)):
                print("+++ wining bought at:"+str(pdata.loc[0,"price"]) +" sold at: "+str(pp) )
                break
        print("--- losing bought at: "+str(pdata.loc[0,"price"]) +" sold at: "+str(pp) )
            
    time.sleep(30)